In [21]:
from dotenv import load_dotenv, find_dotenv
from smolagents import (
    CodeAgent,
    DuckDuckGoSearchTool,
    InferenceClientModel,
    VisitWebpageTool,
    WikipediaSearchTool,
    FinalAnswerTool,
    PythonInterpreterTool,
    Tool,
)
# import nest_asyncio

# nest_asyncio.apply()

_ = load_dotenv(find_dotenv())

In [ ]:
from huggingface_hub import model_info, InferenceClient

In [29]:
from PIL import Image
from io import BytesIO
import base64
from pathlib import Path

In [ ]:
client = InferenceClient(
    provider="hyperbolic",
    bill_to="VitalNest",
)


In [2]:
all_inference_models = !curl -s https://huggingface.co/api/models | jq ".[].id"

In [29]:

info = model_info("google/gemma-3-27b-it", expand="inference")
info.inference


'warm'

In [22]:
vlm_model = InferenceClientModel(model_id="Qwen/Qwen2.5-VL-72B-Instruct",bill_to="VitalNest",
                                 timeout=300,temperature=.1, max_tokens=10000)

In [2]:
def encode_image_to_base64(image_path):
    """
    Encodes an image file to a base64 string.

    Args:
        image_path (str): Path to the image file.

    Returns:
        str: Base64 encoded image string.
    """
    """Encodes a PIL image to a base64 string."""
    image = Image.open(image_path)
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


In [36]:
# MAXIMUM of 4 images can be sent
img_paths = [
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_42.png"
    ),
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_12.png"
    ),
    Path(
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_1.png"
    ),
    # Path(
    #     "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_43.png"
    # ),
    # Path(
    #     "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_44.png"
    # ),
]

In [37]:
# grouped_images = [Image.open(fp=pth) for pth in img_paths]
grouped_images = [encode_image_to_base64(pth) for pth in img_paths]

In [38]:
text_query = """what critical illnesses are covered under optima restore?
If the answer isn't in these documents, say you don't know."""
chat_template = [
    {
"role":"system", "content":"""You find answers from the relevant documents. Answer only 
from these documents. If answer isn't available return 'Question cannot be answered based
on the documents provided.' """
    },
    {
        "role": "user",
        "content": [
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            for image in grouped_images
        ]
        + [{"type": "text", "text": text_query}],
    }
]


In [39]:
print(vlm_model.generate(messages=chat_template).content)

The critical illnesses covered under Optima Restore include Cancer, Open Chest CABG, First Heart Attack, Kidney Failure, Major Organ/Bone Marrow Transplant, Multiple Sclerosis, Permanent Paralysis of Limbs, and Stroke.


In [41]:
completion = client.chat.completions.create(
    model="Qwen/Qwen2.5-VL-72B-Instruct",
    messages=chat_template,
    temperature=.1,
    max_tokens=10_000
)

print(completion.choices[0].message.content)

# completion = client.chat.completions.create(
#     model="Qwen/Qwen2.5-VL-72B-Instruct",
#     messages=[
#         {
#             "role": "user",
#             "content": [
#                 {"type": "text", "text": "Describe the images in one sentence."},
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://cdn.britannica.com/61/93061-050-99147DCE/Statue-of-Liberty-Island-New-York-Bay.jpg"
#                     },
#                 },
#                 {
#                     "type": "image_url",
#                     "image_url": {
#                         "url": "https://upload.wikimedia.org/wikipedia/commons/3/3d/Nicu%C8%99or_Dan_%288_mai_2025%29.jpg"
#                     },
#                 },
#             ],
#         }
#     ],
# )

# print(completion.choices[0].message)


The critical illnesses covered under Optima Restore include Cancer, Open Chest CABG, First Heart Attack, Kidney Failure, Major Organ/Bone Marrow Transplant, Multiple Sclerosis, Permanent Paralysis of Limbs, and Stroke.


In [24]:
class InsuranceInfoRetriever(Tool):
    name = "InsuranceInfoRetriever"
    description = "Retrieves information from insurance documents."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to search for."
        },
        "search_res": {
            "type": "string",
            "description": "Insurance document search results."
        },
        "answer": {
            "type": "string",
            "description": "Answer to the query."
        }
    }
    output_type = "string"

    def forward(self, query: str, search_res:list[str],answer: str ) -> str:
        def encode_image_to_base64(image_path):
            """
            Encodes an image file to a base64 string.

            Args:
                image_path (str): Path to the image file.

            Returns:
                str: Base64 encoded image string.
            """
            """Encodes a PIL image to a base64 string."""
            image = Image.open(image_path)
            buffered = BytesIO()
            image.save(buffered, format="PNG")
            return base64.b64encode(buffered.getvalue()).decode("utf-8")
        img_paths = [Path(res) for res in search_res]
        client = InferenceClient(
            provider="hyperbolic",
            bill_to="VitalNest",
        )
        grouped_images = [encode_image_to_base64(pth) for pth in img_paths]
        chat_template = [
            {
        "role":"system", "content":"""You find answers from the relevant documents. Answer only 
        from these documents. If answer isn't available return 'Question cannot be answered based
        on the documents provided.' """
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{image}"},
                    }
                    for image in grouped_images
                ]
                + [{"type": "text", "text": query}],
            }
        ]
        completion = client.chat.completions.create(
            model="Qwen/Qwen2.5-VL-72B-Instruct",
            messages=chat_template,
            temperature=0.1,
            max_tokens=10_000,
        )
        answer = completion.choices[0].message.content
        return answer if answer else ""

# class SaveContent(Tool):
#     name = "SaveContent"
#     description = "Saves content to a file."
#     inputs = {
#         "content": {
#             "type": "string",
#             "description": "The content to save."
#         },
#         "filename": {
#             "type": "string",
#             "description": "The filename."
#         }
#     }
#     output_type = "string"

#     def forward(self, content: str, filename: str ):
#         with open(filename, "w") as f:
#             f.write(content)
#         return f"Content saved to {filename}"



In [30]:
insurance_agent = CodeAgent(tools=[InsuranceInfoRetriever(), 
                         PythonInterpreterTool(), FinalAnswerTool()], 
                  model=InferenceClientModel(bill_to="VitalNest"),
                  additional_authorized_imports=["os", "requests", "bs4", "pil", "base64", "io"],
                 max_steps=4, planning_interval=2, verbosity_level=0,
                 name="insurance_agent", 
                 description="You answer insurance questions based on the search results provided to you."
                 )

In [ ]:
query = "what critical illnesses are covered under optima restore?"
search_res = [
    (
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_42.png"
    ),
    (
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_12.png"
    ),
    (
        "../src/pages/optima-restore-revision.pdf/optima-restore-revision.pdf_page_1.png"
    ),
]
insurance_agent.run(task=f"""
For the insurance search results below, answer the following query:{query}.
search_res: {search_res}
""")

['Cancer',
 'Open Chest CABG',
 'First Heart Attack',
 'Kidney Failure',
 'Major Organ/Bone Marrow Transplant',
 'Multiple Sclerosis',
 'Permanent Paralysis of Limbs',
 'Stroke']

In [2]:
from smolagents import ToolCallingAgent

In [9]:
websearch_agent = ToolCallingAgent(
    model=InferenceClientModel(
        model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest", temperature=0.1
    ),
    tools=[
        VisitWebpageTool(max_output_length=20_000),
        DuckDuckGoSearchTool(max_results=5),
    ],
    max_steps=4,
    verbosity_level=0,
    name="web_search_agent",
    planning_interval=2,
    description="Searches the web with a particular query.",
)

In [ ]:
# websearch_agent.run("what's multi head latent attention?")

In [11]:
wikipedia_agent = ToolCallingAgent(
    model=InferenceClientModel(model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest", temperature=.1),
    tools=[WikipediaSearchTool(user_agent="WikiAssistant (merlin@example.com)")],
    max_steps=3,
    verbosity_level=0,
    name="wikipedia_agent",
    description="Searches Wikipedia for a topic.",
)

In [34]:
manager_agent = CodeAgent(
    tools=[PythonInterpreterTool(), FinalAnswerTool()],
    additional_authorized_imports=["os", "requests", "bs4"],
    model=InferenceClientModel(
        model_id="Qwen/Qwen3-30B-A3B", bill_to="VitalNest", temperature=0.1
    ),  # "Qwen/Qwen3-235B-A22B"
    managed_agents=[websearch_agent, wikipedia_agent, insurance_agent],
    max_steps=10,
    planning_interval=3,
    verbosity_level=0,
)

In [35]:
manager_agent.run(
    task=f"""
For the insurance search results below, answer the following query:{query}.
search_res: {search_res}
"""
    #     """Search for `topic`:multi head latent attention in wikipedia.
    #                   If there's no information about the same in Wikipedia, search the web
    #                   for it and return a summary of the findings along with the search result links.
    # """
)

Code execution failed at line 'insurance_response = insurance_agent(task="What critical illnesses are covered under
Optima Restore? Please analyze the provided PDF pages: " + ", ".join(search_res))' due to: InterpreterError: The 
variable `search_res` is not defined.

Code execution failed at line 'search_results = web_search_agent(query=search_query)' due to: TypeError: 
MultiStepAgent.__call__() missing 1 required positional argument: 'task'

Code execution failed at line 'import pytesseract' due to: InterpreterError: Import of pytesseract is not allowed. 
Authorized imports are: ['io', 'requests', 'queue', 'random', 'statistics', 'datetime', 'collections', 'math', 
'stat', 'unicodedata', 'os', 'bs4', 're', 'base64', 'pil', 'time', 'itertools'\]

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\s*\n(.*?)\n``` was not found in it.
                Here is your code snippet:
                <think>
Okay, let's see. The user asked about the critical illnesses covered under Optima Restore, and the insurance_agent 
provided a detailed list. The response from the insurance_agent includes a list of conditions like Cancer, Heart 
Attack, Stroke, Kidney Failure, etc. The user's search_res was the list of PDF pages, and after defining search_res
properly, the insurance_agent analyzed them and returned the answer.

Now, the next step is to present the final answer. The insurance_agent's response has the list, so I need to 
extract that and use the final_answer tool. The answer is the list of critical illnesses mentioned. I should check 
if the response is correctly formatted and then output it.

Wait, the observation shows that the insurance_agent's response has a 'Task outcome (short version)' with the list.
So, the code should take that and pass it to final_answer. But since the code is in the python_interpreter, I need 
to capture that output. However, in the previous step, the code printed the insurance_response, but the observation
shows that the insurance_agent's answer is already provided. 

So, the final answer is the list of illnesses from the insurance_agent's response. The user's task is to answer the
query, so the final answer should be the list of conditions. Therefore, the correct code is to call final_answer 
with the list from the insurance_response. But since the code is in the python_interpreter, I need to make sure 
that the code uses the insurance_response variable. However, in the previous step, the code printed the 
insurance_response, but the observation shows that the insurance_agent's answer is already there. 

Wait, the user's last code call was to the insurance_agent, and the observation includes the response. Now, the 
assistant needs to take that response and format the final answer. Since the insurance_agent's response includes 
the list, the final answer can be constructed from that. 

So, the code should be to call final_answer with the list of critical illnesses. The exact list is in the 'Task 
outcome (short version)' part. The answer should be a string listing those illnesses. 

Therefore, the final code would be to print the list or directly pass it to final_answer. But since the user's 
previous code printed the insurance_response, and the observation shows the answer, the next step is to use that 
data. 

So, the correct final answer is the list of conditions provided by the insurance_agent. The assistant should format
that into a coherent answer.
</think>

The insurance_agent has analyzed the provided PDF pages and identified the critical illnesses covered under Optima 
Restore. The list includes conditions such as Cancer, Heart Attack, Stroke, Kidney Failure, and others. 

**Final Answer**
The critical illnesses covered under Optima Restore include Alzheimer's Disease, Aorta Surgery, Aplastic Anemia, 
Cancer, Chronic Total Ischemia or acidosis, Diabetes with Complications, Heart Attack, Heart Valve Replacement 
Surgery, Kidney Failure (Renal Failure), Major Organ Transplant, Major complication and permanent disability 
following Coronary Artery Bypass Surgery, Multiple Sclerosis, Myocardial Infarction, Paralysis from CVA or Stroke, 
Paralysis from Traumatic Injury, Parkinson's Disease, Severe Active Rheumatoid Arthritis, Severe Dementia, Severe 
Lower Limb Transplant, Severe Sickle Cell Anemia, Stroke, Systemic Lupus Erythematosus Deforming, and others. For 
precise details, refer to the policy's Terms and Conditions.


                It seems like you're trying to return the final answer, you can do it as follows:
                Code:
                ```py
                final_answer("YOUR FINAL ANSWER HERE")
                ```<end_code>
Make sure to provide correct code blobs.

"Alzheimer's Disease, Aorta Surgery, Aplastic Anemia, Cancer, Chronic Total Ischemia or acidosis, Diabetes with Complications, Heart Attack, Heart Valve Replacement Surgery, Kidney Failure (Renal Failure), Major Organ Transplant, Major complication and permanent disability following Coronary Artery Bypass Surgery, Multiple Sclerosis, Myocardial Infarction, Paralysis from CVA or Stroke, Paralysis from Traumatic Injury, Parkinson's Disease, Severe Active Rheumatoid Arthritis, Severe Dementia, Severe Lower Limb Transplant, Severe Sickle Cell Anemia, Stroke, Systemic Lupus Erythematosus Deforming, and others."

In [1]:
import torch
from PIL import Image
from transformers.utils.import_utils import is_flash_attn_2_available

from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor


In [ ]:

model = ColQwen2_5.from_pretrained(
    pretrained_model_name_or_path="vidore/colqwen2.5-v0.2",
    torch_dtype=torch.bfloat16,
    device_map="cuda:0",  # or "mps" if on Apple Silicon
    attn_implementation="flash_attention_2" if is_flash_attn_2_available() else None,
).eval()
processor = ColQwen2_5_Processor.from_pretrained("vidore/colqwen2.5-v0.2", use_fast=True)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:

# Your inputs
images = [
    Image.new("RGB", (128, 128), color="white"),
    Image.new("RGB", (128, 128), color="black"),
]
queries = [
    "Is attention really all you need?",
    "What is the amount of bananas farmed in Salvador?",
]

# Process the inputs
batch_images = processor.process_images(images).to(model.device)
batch_queries = processor.process_queries(queries).to(model.device)

# Forward pass
with torch.no_grad():
    image_embeddings = model(**batch_images)
    query_embeddings = model(**batch_queries)

scores = processor.score_multi_vector(query_embeddings, image_embeddings)
